# Tutorial 13: Best Practices and Advanced Topics

In this tutorial, we'll cover best practices and advanced topics for developing and deploying LangChain and LangGraph applications. We'll explore performance optimization, handling rate limits and API costs, security considerations, deployment strategies, and monitoring and logging in production.

## Setup

First, let's import the necessary libraries and set up our environment:

In [1]:
import os
import asyncio
from typing import List, Dict, Any
from langchain_groq import ChatGroq
from langchain.callbacks import get_openai_callback
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.cache import InMemoryCache
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from prometheus_client import Counter, Histogram

# Initialize Groq LLM
llm = ChatGroq(
        model_name="llama-3.1-70b-versatile",
        temperature=0.7,
        model_kwargs={"top_p": 0.8, "seed": 1337}
    )

## 1. Performance optimization techniques

Let's explore some performance optimization techniques for LangChain applications:

In [2]:
# Enable caching
import langchain
langchain.llm_cache = InMemoryCache()

# Create an async version of the LLM
async_llm = llm.agenerate

# Define a simple prompt template
prompt = PromptTemplate(
    input_variables=["topic"],
    template="Write a short paragraph about {topic}."
)

# Create an LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

# Function to process topics sequentially
def process_topics_sequential(topics: List[str]) -> List[str]:
    return [chain.run(topic) for topic in topics]

# Function to process topics asynchronously
async def process_topics_async(topics: List[str]) -> List[str]:
    async_chain = LLMChain(llm=async_llm, prompt=prompt)
    tasks = [async_chain.arun(topic) for topic in topics]
    return await asyncio.gather(*tasks)

# Compare performance
topics = ["Python", "Machine Learning", "Artificial Intelligence", "Data Science", "Web Development"]

# Sequential processing
%time sequential_results = process_topics_sequential(topics)

# Asynchronous processing
%time async_results = await process_topics_async(topics)

print(f"Sequential processing time: {time_sequential}")
print(f"Asynchronous processing time: {time_async}")

/tmp/ipykernel_213011/1461860608.py:15: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  chain = LLMChain(llm=llm, prompt=prompt)
/tmp/ipykernel_213011/1461860608.py:19: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  return [chain.run(topic) for topic in topics]


CPU times: user 87.4 ms, sys: 7 ms, total: 94.4 ms
Wall time: 4.38 s


SyntaxError: 'await' outside function (<timed exec>, line 1)

## 2. Handling rate limits and API costs

Let's implement a system to handle rate limits and track API costs:

In [16]:
from langchain.callbacks import get_openai_callback
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

# Note: OpenAI is used here as an example. Adjust for Groq as needed.
# Initialize Groq LLM
chat_model = ChatGroq(
        model_name="llama-3.1-70b-versatile",
        temperature=0.7,
        model_kwargs={"top_p": 0.8, "seed": 1337}
    )

def run_with_cost_tracking(prompt: str) -> Dict[str, Any]:
    with get_openai_callback() as cb:
        response = chat_model([HumanMessage(content=prompt)])
        print (response)
    return {
        "response": response.content,
        "total_tokens": cb.total_tokens,
        "prompt_tokens": cb.prompt_tokens,
        "completion_tokens": cb.completion_tokens,
        "total_cost": cb.total_cost,
    }

# Example usage
result = run_with_cost_tracking("Explain the concept of machine learning in one paragraph.")
print(f"Response: {result['response']}")
print(f"Total tokens: {result['total_tokens']}")
print(f"Total cost: ${result['total_cost']:.4f}")

content='Machine learning is a subset of artificial intelligence that involves training algorithms to learn patterns and make predictions or decisions based on data. It works by feeding large amounts of data to a computer system, which then uses statistical techniques to identify relationships and trends within the data. The algorithm can then use this learned knowledge to make predictions or classify new, unseen data. Unlike traditional programming, where a computer is explicitly told what to do, machine learning allows the system to learn and improve on its own through experience, making it a powerful tool for tasks such as image recognition, natural language processing, and predictive analytics.' response_metadata={'token_usage': {'completion_tokens': 119, 'prompt_tokens': 46, 'total_tokens': 165, 'completion_time': 0.476, 'prompt_time': 0.014790308, 'queue_time': 0.005393032000000001, 'total_time': 0.490790308}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_926

## 3. Security considerations

When working with LangChain and LangGraph applications, consider the following security best practices:

1. Use environment variables for API keys and sensitive information
2. Implement input validation and sanitization
3. Use HTTPS for all API communication
4. Implement proper authentication and authorization
5. Regularly update dependencies
6. Be cautious with user-provided content in prompts

Here's an example of input validation:

In [19]:
from pydantic import BaseModel, Field, field_validator

class UserInput(BaseModel):
    prompt: str = Field(..., min_length=1, max_length=1000)
    
    @field_validator('prompt')
    def no_sensitive_info(cls, v):
        sensitive_words = ['password', 'credit card', 'social security']
        if any(word in v.lower() for word in sensitive_words):
            raise ValueError("Input contains sensitive information")
        return v

# Example usage
try:
    user_input = UserInput(prompt="Tell me about AI")
    print("Valid input:", user_input)
    
    invalid_input = UserInput(prompt="My password is 123456")
except ValueError as e:
    print("Invalid input:", str(e))

Valid input: prompt='Tell me about AI'
Invalid input: 1 validation error for UserInput
prompt
  Value error, Input contains sensitive information [type=value_error, input_value='My password is 123456', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error


## 4. Deploying LangChain and LangGraph applications

Let's create a simple FastAPI application that uses our LangChain model:

In [20]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel

app = FastAPI()

class Query(BaseModel):
    text: str

@app.post("/generate")
async def generate_text(query: Query):
    try:
        result = await chain.arun(query.text)
        return {"generated_text": result}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# To run the FastAPI app, use the following command in your terminal:
# uvicorn main:app --reload

# Note: This cell won't actually start the server in the notebook.
# It's meant to be run as a separate Python file.

## 5. Monitoring and logging in production

Let's implement basic monitoring using Prometheus metrics:

In [21]:
from prometheus_client import Counter, Histogram

# Define Prometheus metrics
REQUESTS = Counter('api_requests_total', 'Total API requests')
LATENCY = Histogram('api_latency_seconds', 'API latency')

# Update the FastAPI endpoint to use metrics
@app.post("/generate")
async def generate_text(query: Query):
    REQUESTS.inc()
    with LATENCY.time():
        try:
            result = await chain.arun(query.text)
            return {"generated_text": result}
        except Exception as e:
            raise HTTPException(status_code=500, detail=str(e))

# Add a metrics endpoint
@app.get("/metrics")
async def metrics():
    from prometheus_client import generate_latest
    return generate_latest()

# Note: This cell won't actually start the server in the notebook.
# It's meant to be run as a separate Python file.

## Conclusion

In this tutorial, we've covered several best practices and advanced topics for LangChain and LangGraph applications:

1. Performance optimization techniques, including caching and asynchronous processing
2. Handling rate limits and tracking API costs
3. Security considerations and input validation
4. Deploying applications using FastAPI
5. Monitoring and logging in production using Prometheus metrics

These practices will help you build more efficient, secure, and production-ready AI applications using LangChain and LangGraph.

## Next Steps

To further improve your LangChain and LangGraph applications:

1. Implement more advanced caching strategies, such as using Redis for distributed caching
2. Explore containerization (e.g., Docker) for easier deployment and scaling
3. Implement more comprehensive logging and error handling
4. Set up CI/CD pipelines for automated testing and deployment
5. Explore advanced monitoring and alerting systems
6. Consider using a reverse proxy (e.g., Nginx) for load balancing and additional security
7. Implement rate limiting and request throttling to protect your API

Remember to always follow best practices for security, performance, and scalability as you develop and deploy your AI applications